# EDSA - Climate Change Belief Analysis 2022
# TeamNM6- 2201ACDS_ NM6

### Notebook created by (June 2022)

- #### Samuel Njoki
- #### Mohamed Abubakar
- #### Ubong Ben
- #### Yinka Akindele
- #### Segun Lawal




# Introduction

## Climate Change Based On Novel Tweet Data




## Problem Statement

Increase Companies Market Research/Advertising Efficiency using Machine Learning to create Marketing tools that can identify whether or not a person believes in climate change and could possibly be converted to a new customer based on their tweets.



<img src="https://i.gifer.com/RD07.gif">

# Table of Contents:
- [Import important libraries and datasets](#section-one)
- [A look at our datasets](#section-two)
- [Data Pre-processing](#section-three)


# Start Comet Experiment
We will be using Comet as version control throughout the development of our model

In [1]:
"""
!pip install comet_ml
from comet_ml import Experiment

# Setting the API key (saved as environment variable)
experiment = Experiment(api_key="THysD8zqvW8wCiFTidV67jLP2",
                        project_name="climate-change-belief-analysis", 
                        workspace="jamakasilwane")                      
"""

'\n!pip install comet_ml\nfrom comet_ml import Experiment\n\n# Setting the API key (saved as environment variable)\nexperiment = Experiment(api_key="THysD8zqvW8wCiFTidV67jLP2",\n                        project_name="climate-change-belief-analysis", \n                        workspace="jamakasilwane")                      \n'

<a id="section-one"></a>
# Importing important libraries and datasets
We first need to load the libraries we are going to use throughout our notebook. Then load our train and test data sets

In [29]:
conda install -c conda-forge wordcloud



Note: you may need to restart the kernel to use updated packages.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: wordcloud


In [30]:
from wordcloud import WordCloud

In [2]:
# Standard libraries
import re
import csv
import nltk
# import spacy
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Style
import matplotlib.style as style 
sns.set(font_scale=1.5)
style.use('seaborn-pastel')
style.use('seaborn-poster')
from PIL import Image
from wordcloud import WordCloud

# Downloads
# nlp = spacy.load('en')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Preprocessing
# import en_core_web_sm
from collections import Counter
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, wordnet  
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Building classification models
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

[nltk_data] Downloading package punkt to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Segun Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# Import and load dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test_with_no_labels.csv')
sample = pd.read_csv('sample_submission.csv')

<a id="section-two"></a>
# A look at our datasets

In [4]:
# a look at the train and test datasets

print(train.shape)
print(test.shape)

display(train.head())
display(test.head())

percentage_duplicate = round(
    (1 - (train['message'].nunique() / len(train['message']))) * 100, 2)

print("{}% of tweets in train dataset are duplicate".format(
    percentage_duplicate))

(15819, 3)
(10546, 2)


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


10.05% of tweets in train dataset are duplicate


In [5]:
# a look at the unique values in the label column of the train dateframe
train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

The train dataset contains over 15,000 tweets while the test dataset contains over 10,000 tweets.

The tweets are divided into 4 classes:

* [ 2 ] News : Tweets linked to factual news about climate change.

* [ 1 ] Pro : Tweets that support the belief of man-made climate change.

* [ 0 ] Neutral : Tweets that neither support nor refuse beliefs of climate change.

*  [-1 ] Anti : Tweets that do not support the belief of man-made climate change.

10% of the messages in the train dataset were duplicated.

Note: This duplicated messages are a called Retweets 

<a id="section-three"></a>
# Data Pre-processing
We will be preparing (cleaning and organizing) our raw data before having an in-depth Exploratory Data Analysis on the cleansed data. 

In [7]:
def reform(df):
    
    """
    This fuction creates a copy of the original train data, 
    converting the classes from numbers to their respective meanings
    in words.
    
    Input:
    
    df: original dataframe
        datatype: dataframe
        
    Output:
    
    df: modified dataframe
        datatype: dataframe
        
    """
    
    df = train.copy()
    sentiment = df['sentiment']
    word_sentiment = []
    
    for word in sentiment:
        if word == -1:
            word_sentiment.append('Anti')
        elif word == 0:
            word_sentiment.append('Neutral')
        elif word == 1:
            word_sentiment.append('Pro')
        else:
            word_sentiment.append('News')
            
    df['sentiment'] =  word_sentiment
    
    return df

df = reform(train)
df.head()

,sentiment,message,tweetid
0,Pro,PolySciMajor EPA chief doesn't think carbon di...,625221
1,Pro,It's not like we lack evidence of anthropogeni...,126103
2,News,RT @RawStory: Researchers say we have three ye...,698562
3,Pro,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,Pro,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


## Extract Hashtag

In [18]:
def extract_hashtag(tweet):
    
    """
    This function takes in a tweet and extracts the top 15 hashtag(s) 
    using regular expressions.
    These hashtags are stored in a seperate dataframe along with a 
    count of how frequently the occur
    
    Input:
    
    tweet: original tweets
           datatype: 'str'
    
    Output:
    df_hashtag: dataframe containing the top hashtags in the tweets
                datatype: dataframe
    
    """
    
    hashtags = []
    
    for i in tweet:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)
        
    hashtags = sum(hashtags, [])
    frequency = nltk.FreqDist(hashtags)
    
    hashtag_df = pd.DataFrame({'hashtag': list(frequency.keys()),
                       'count': list(frequency.values())})
    hashtag_df = hashtag_df.nlargest(15, columns="count")

    return hashtag_df

# Extract the hashtag from tweets in each class
pro = extract_hashtag(df['message'][df['sentiment'] == 'Pro'])
anti = extract_hashtag(df['message'][df['sentiment'] == 'Anti'])
neutral = extract_hashtag(df['message'][df['sentiment'] == 'Neutal'])
news = extract_hashtag(df['message'][df['sentiment'] == 'News'])
                        
pro.head()

,hashtag,count
19,climate,187
24,BeforeTheFlood,129
68,climatechange,94
13,ImVotingBecause,62
4,COP22,59


## Cleaning Tweet

In [19]:
def clean_tweet(tweet):
    
    # Convert tweet to lowercase
    tweet = tweet.lower()
    
    # Remove mentions
    tweet = re.sub('@[\w]*', '', tweet)
    
    # Rwmove url's
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    
    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)
    
    # Remove punctuations
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    
    # Remove that funny diamond
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    
    # Remove extra whitespace
    tweet = re.sub(r'\s\s+', ' ', tweet)
    
    # Remove space in front of tweet
    tweet = tweet.lstrip(' ')                        
    
    return tweet

# Clean th tweets in the message column

df['message'] = df['message'].apply(clean_tweet)

df.head()

,sentiment,message,tweetid
0,Pro,polyscimajor epa chief doesn t think carbon di...,625221
1,Pro,it s not like we lack evidence of anthropogeni...,126103
2,News,rt researchers say we have three years to act ...,698562
3,Pro,wired was a pivotal year in the war on climate...,573736
4,Pro,rt it s and a racist sexist climate change den...,466954


## Parts of speech tagging and lemmatization

In [22]:
def lemmatize(df):
    
    
    df['length'] = df['message'].str.len()
    df['tokenized'] = df['message'].apply(word_tokenize)
    df['pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag)
    
    
    def get_wordnet_pos(tag):

        if tag.startswith('J'):
            return wordnet.ADJ

        elif tag.startswith('V'):
            return wordnet.VERB

        elif tag.startswith('N'):
            return wordnet.NOUN

        elif tag.startswith('R'):
            return wordnet.ADV
    
        else:
            return wordnet.NOUN
    
    wnl = WordNetLemmatizer()
    df['pos_tags'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
    df['lemmatized'] = df['pos_tags'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
    df['lemmatized'] = [' '.join(map(str, l)) for l in df['lemmatized']]  
    return df


df = lemmatize(df)
df.head()

,sentiment,message,tweetid,length,tokenized,pos_tags,lemmatized
0,Pro,polyscimajor epa chief doesn t think carbon di...,625221,102,"[polyscimajor, epa, chief, doesn, t, think, ca...","[(polyscimajor, a), (epa, n), (chief, n), (doe...",polyscimajor epa chief doesn t think carbon di...
1,Pro,it s not like we lack evidence of anthropogeni...,126103,62,"[it, s, not, like, we, lack, evidence, of, ant...","[(it, n), (s, v), (not, r), (like, n), (we, n)...",it s not like we lack evidence of anthropogeni...
2,News,rt researchers say we have three years to act ...,698562,86,"[rt, researchers, say, we, have, three, years,...","[(rt, n), (researchers, n), (say, v), (we, n),...",rt researcher say we have three year to act on...
3,Pro,wired was a pivotal year in the war on climate...,573736,54,"[wired, was, a, pivotal, year, in, the, war, o...","[(wired, v), (was, v), (a, n), (pivotal, a), (...",wire be a pivotal year in the war on climate c...
4,Pro,rt it s and a racist sexist climate change den...,466954,81,"[rt, it, s, and, a, racist, sexist, climate, c...","[(rt, v), (it, n), (s, n), (and, n), (a, n), (...",rt it s and a racist sexist climate change den...


## Word frequency

In [23]:
def frequency(tweet):
    
    """
    This function determines the frequency of each word in a collection of tweets 
    and stores the 25 most frequent words in a dataframe, 
    sorted from most to least frequent
    
    Input: 
    tweet: original tweets
           datatype: 'str'
           
    Output: 
    frequency: dataframe containing the top 25 words 
               datatype: dataframe          
    """
    
    # Count vectorizer excluding english stopwords
    cv = CountVectorizer(stop_words='english')
    words = cv.fit_transform(tweet)
    
    # Count the words in the tweets and determine the frequency of each word
    sum_words = words.sum(axis=0)
    words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    
    # Create a dataframe to store the top 25 words and their frequencies
    frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])
    frequency = frequency.head(25)
    
    return frequency

# Extract the top 25 words in each class
pro_frequency = frequency(df['lemmatized'][df['sentiment']=='Pro'])
anti_frequency = frequency(df['lemmatized'][df['sentiment']=='Anti'])
news_frequency = frequency(df['lemmatized'][df['sentiment']=='News'])
neutral_frequency = frequency(df['lemmatized'][df['sentiment']=='Neutral'])

In [31]:
# Extract the words in the tweets for the pro and anti climate change classes 
anti_words = ' '.join([text for text in anti_frequency['word']])
pro_words = ' '.join([text for text in pro_frequency['word']])
news_words = ' '.join([text for text in news_frequency['word']])
neutral_words = ' '.join([text for text in neutral_frequency['word']])

# Create wordcloud for the anti climate change class
anti_wordcloud = WordCloud(width=800, 
                           height=500, 
                           random_state=110, 
                           max_font_size=110, 
                           background_color='white',
                           colormap="Reds").generate(anti_words)

# Create wordcolud for the pro climate change class
pro_wordcloud = WordCloud(width=800, 
                          height=500, 
                          random_state=73, 
                          max_font_size=110, 
                          background_color='white',
                          colormap="Greens").generate(pro_words)

# Create wordcolud for the news climate change class
news_wordcloud = WordCloud(width=800, 
                          height=500, 
                          random_state=0, 
                          max_font_size=110, 
                          background_color='white',
                          colormap="Blues").generate(news_words)

# Create wordcolud for the neutral climate change class
neutral_wordcloud = WordCloud(width=800, 
                          height=500, 
                          random_state=10, 
                          max_font_size=110, 
                          background_color='white',
                          colormap="Oranges").generate(neutral_words)

pro_frequency.tail()

,word,freq
20,http,359
21,make,322
22,like,318
23,warm,313
24,husband,312


# Exploratory data analysis 